In [1]:
from bs4 import BeautifulSoup
import re
import sqlite3

# Парсинг person
## TODO:
На некоторых страницах почему-то не выводится опыт работы. Например на 8 записи вместо 138 выводится 0!

In [2]:
database = "data.db"

def insert_person_data(conn, data):
    sql = ''' INSERT INTO person (age, gender, town, key_profession, specialtys, key_skills, months_of_exp)
              VALUES (?, ?, ?, ?, ?, ?, ?) '''
    cur = conn.cursor()
    cur.execute(sql, data)
    conn.commit()
    return cur.lastrowid

In [3]:
for i in range(1, 1002):
    with open(f'../data/resumes/resume ({i}).html', 'r', encoding='utf-8') as file:
        html_content = file.read()
    
    soup = BeautifulSoup(html_content, 'html.parser')

    # -------------------------
    # Пол, возраст, город
    gender_element = soup.find('span', {'data-qa': 'resume-personal-gender'})
    gender = gender_element.text if gender_element else None
    
    age_element = soup.find('span', {'data-qa': 'resume-personal-age'})
    age_text = age_element.text if age_element else None
    
    city_element = soup.find('span', {'data-qa': 'resume-personal-address'})
    city = city_element.text if city_element else None
    
    # Используем регулярные выражения для извлечения числа из строки возраста
    if age_text:
        age = int(re.search(r'\d+', age_text).group())
    else:
        age = None
    
    # -------------------------
    # Специализации
    specializations_elements = soup.find_all('li', class_='resume-block__specialization')
    
    specializations = [element.text.strip() for element in specializations_elements]
    
    specializations = ', '.join(specializations)

    # -------------------------
    # Опыт работы
    experience_elements = soup.find_all('span', class_='resume-block__title-text_sub')[0].find_all('span')

    years = 0
    months = 0
    
    for element in experience_elements:
        text = element.text.strip()
        # Проверяем, содержит ли текст информацию о годах
        if 'лет' in text or 'год' in text:
            years = int(re.search(r'\d+', text).group())
    
        # Проверяем, содержит ли текст информацию о месяцах
        elif 'месяц' in text or 'месяцев' in text:
            months = int(re.search(r'\d+', text).group())
    
    total_months = years * 12 + months

    # -------------------------
    # Ключевые навыки
    if "Ключевые навыки" in html_content:
        # Находим блок с ключевыми навыками
        skills_block = soup.find('div', class_='bloko-tag-list')
        
        # Находим все элементы с ключевыми навыками
        skill_elements = skills_block.find_all('div', {'data-qa': 'bloko-tag bloko-tag_inline'})
        
        # Извлекаем текст каждого навыка
        skills = [skill.find('span', {'data-qa': 'bloko-tag__text'}).text.strip() for skill in skill_elements]
        
        skills = ', '.join(skills)
    else:
        skills = None

    # -------------------------
    # Основная профессия
    profession_element = soup.find('span', class_='resume-block__title-text')
    profession = profession_element.text.strip()
    profession

    # -------------------------
    # Запись в БД
    conn = sqlite3.connect(database)
    
    data = (age, gender, city, profession, specializations, skills, total_months)
    insert_person_data(conn, data)
    
    conn.close()

# Парсинг образования

In [8]:
def read_resume_file(file_id):
    with open(f'../data/resumes/resume ({file_id}).html', 'r', encoding='utf-8') as file:
        return file.read()

def process_html(html_content, soup, person_id):
    is_high = 1 if "Высшее образование" in html_content else 0
    education_blocks = soup.find_all('div', {'data-qa': 'resume-block-education-item'})

    if not education_blocks:
        # Обработка случаев, когда блоки образования отсутствуют
        return [(person_id, None, None, is_high)]
    
    education_data = []
    for block in education_blocks:
        university_element = block.find('div', {'data-qa': 'resume-block-education-name'})
        university_name = university_element.text.strip() if university_element else 'Не указано'

        specialization_element = block.find('div', {'data-qa': 'resume-block-education-organization'})
        specialization_text = specialization_element.text.strip() if specialization_element else 'Не указано'

        education_data.append((person_id, university_name, specialization_text, is_high))
    
    return education_data


def main():
    with sqlite3.connect('data.db') as conn:
        cursor = conn.cursor()
        all_data = []

        for i in range(1, 1002):
            html_content = read_resume_file(i)
            soup = BeautifulSoup(html_content, 'html.parser')
            education_data = process_html(html_content, soup, i)
            all_data.extend(education_data)

        cursor.executemany("INSERT INTO education (person_id, name, specialitys, is_high_edu) VALUES (?, ?, ?, ?)", all_data)
        conn.commit()

if __name__ == "__main__":
    main()

# Парсинг jobs
## TODO: починить

In [1]:
# Находим основной блок с опытом работы
experience_main_block = soup.find('div', {'data-qa': 'resume-block-experience'})

if experience_main_block:
    # Находим все блоки с датами работы
    date_blocks = experience_main_block.find_all('div', class_='bloko-column')

    experiences = []

    for date_block in date_blocks:
        # Извлекаем соответствующий блок работы
        work_block = date_block.find_next('div', class_='resume-block-container')

        if work_block:
            position_element = work_block.find('div', {'data-qa': 'resume-block-experience-position'})
            position = position_element.text.strip() if position_element else 'Не указано'

            description_element = work_block.find('div', {'data-qa': 'resume-block-experience-description'})
            description = description_element.text.strip() if description_element else 'Описание отсутствует'

            # Извлекаем даты начала и окончания работы
            date_text = date_block.get_text(strip=True)
            start_date_search = re.search(r'\w+\s\d+', date_text)
            end_date_search = re.search(r'—\s(.+)', date_text)

            start_date = start_date_search.group() if start_date_search else 'Не указано'
            end_date = end_date_search.group(1) if end_date_search else 'Null'

            experiences.append({
                'Название работы': position,
                'Описание': description,
                'Начало работы': start_date,
                'Конец работы': end_date
            })

    for experience in experiences:
        print(experience)
else:
    print('Блок с опытом работы не найден')

NameError: name 'soup' is not defined